# Worksheet 08

Name:  Chris Chan   
UID: U31827126

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.028980705979166, 4.965722317795994, 6.0702841852184575, 4.978673442983568, 4.404593218633171, 6.442191107233376, 4.609036786727812, 4.388552837738249, 5.867883597213469, 4.0451097218901495]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[7.792442134722227, 7.80544864800378, 8.329755893120607, 7.959286571598049, 7.808110950875414, 7.8494887714031565, 9.72591102188842, 7.459315349632037, 7.521898389871084, 8.032875319631511]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
true_assignment = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    true_assignment.append(coin_output)
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[8.032875319631511, 7.521898389871084, 4.0451097218901495, 5.867883597213469, 4.388552837738249, 4.609036786727812, 6.442191107233376, 7.459315349632037, 9.72591102188842, 7.8494887714031565]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

mean1 = 5 \
mean2 = 8 \
stdev1 = 1 \
stdev2 = 1 


d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x: (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

C:\Users\chris\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[4.0451097218901495, 5.867883597213469, 4.388552837738249, 4.609036786727812]
[8.032875319631511, 7.521898389871084, 6.442191107233376, 7.459315349632037, 9.72591102188842, 7.8494887714031565]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.72764573589242,  mean_2 = 7.838613326609932
var_1 = 0.47376246680015854,  var_2 = 0.9656531515162036


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

compute_posterior = lambda x, mu, vars: norm.pdf(x, mean[0], var[0]) 

def compute_marg(x, means, vars, prob_s):
    marg = [norm.pdf(x, mu, var) * p_s for mu, var, p_s in zip(means, vars, prob_s)]
    return sum(marg)

prob_x = [compute_marg(x, mean, var, prob_s) for x in data] # P(X_i)
prob_s0_x = [norm.pdf(x, mean[0], var[0]) * prob_s[0] / p_x for x, p_x in zip(data, prob_x)] # P(S_0 | X_i)
prob_s1_x = [norm.pdf(x, mean[1], var[1]) * prob_s[1] / p_x for x, p_x in zip(data, prob_x)] # P(S_1 | X_i)

k = 2

# for p in data:
#     print("point = ", p)
#     pdf_i = []

#     for j in range(k):
#         # P(X_i | S_j)
#         pdf_i.append(norm.pdf(p, mean[j], var[j]))
#         print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
#         # P(S_j) already computed
#         prob_s[j]

#     # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
#     prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

#     # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    # prob_s0_x.append( ... )
    # prob_s1_x.append( ... )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


8.032875319631511
Probability of coming from S_1 = 3.7402388906047415e-11
Probability of coming from S_2 = 0.9999999999625976

7.521898389871084
Probability of coming from S_1 = 4.006337394974497e-08
Probability of coming from S_2 = 0.999999959936626

4.0451097218901495
Probability of coming from S_1 = 0.9990753397177139
Probability of coming from S_2 = 0.0009246602822861045

5.867883597213469
Probability of coming from S_1 = 0.3758605584246127
Probability of coming from S_2 = 0.6241394415753873

4.388552837738249
Probability of coming from S_1 = 0.9983947119641284
Probability of coming from S_2 = 0.001605288035871646

4.609036786727812
Probability of coming from S_1 = 0.9971793410978903
Probability of coming from S_2 = 0.002820658902109641

6.442191107233376
Probability of coming from S_1 = 0.005506250370093537
Probability of coming from S_2 = 0.9944937496299066

7.459315349632037
Probability of coming from S_1 = 8.860672628293933e-08
Probability of coming from S_2 = 0.999999911393273

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([prob * x for prob, x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([prob * (x - mean[0]) ** 2 for prob, x in zip(prob_s0_x, data)]) , sum([prob * (x - mean[1]) ** 2 for prob, x in zip(prob_s1_x, data)])]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 4.520088582583565,  mean_2 = 7.651342841829325
var_1 = 0.9536735330219897,  var_2 = 8.036728674914986


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_x = [compute_marg(x, mean, var, prob_c) for x in data] # P(X_i)
prob_s0_x = [norm.pdf(x, mean[0], var[0]) * prob_c[0] / p_x for x, p_x in zip(data, prob_x)] # P(S_0 | X_i)
prob_s1_x = [norm.pdf(x, mean[1], var[1]) * prob_c[1] / p_x for x, p_x in zip(data, prob_x)] # P(S_1 | X_i)

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [10]:
assignments = [int(p_s1_x > p_s0_x) for p_s0_x, p_s1_x in zip(prob_s0_x, prob_s1_x)]
print(assignments)
print(true_assignment)

[1, 1, 0, 0, 0, 0, 1, 1, 1, 1]
[1, 1, 0, 0, 0, 0, 0, 1, 1, 1]
